# このノートブックで行っていること
- ポケモン図鑑のクローリング
- 手順
    - ポケモン図鑑から各ポケモンのページリンクを取得
    - ポケモンページリンクをもとにページ取得

In [1]:
import json
import os
import requests
import time

from bs4 import BeautifulSoup
from pathlib import Path
from google.colab import drive
import numpy as np
import pandas as pd
import shutil

pd.set_option('display.max_columns', None)

In [2]:
# ディレクトリ作成
os.makedirs("/content/data/pokemon", exist_ok=True)

# ポケモン作業用ディレクトリを指定
poke_dir = Path("/content/data/pokemon")

# Google Drive をマウント
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ポケモン図鑑から各ポケモンのページリンクを取得
with open("/content/data/pokemon_list.html", "r") as f:
    pokemon_zukan = f.read()

print(type(pokemon_zukan))
print(pokemon_zukan[:400])

<class 'str'>
<html lang="ja" prefix="og: http://ogp.me/ns#"><head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <title>トップページ｜ポケモンずかん</title>
    <meta name="keywords" content="ポケモン,ぽけもん,ポケットモンスター,図鑑,ずかん,ポケモンずかん,ぽけもんずかん">
    <meta name="description" content="『ポケットモンスター』シリーズに登


In [ ]:
# ポケモン図鑑から各ページへのリンクを取得
soup = BeautifulSoup(pokemon_zukan, "html.parser")
mainlist = soup.find("ul", id="mainlist")
pokemon_list = mainlist.find_all("li")
print("ポケモンのページ数", len(pokemon_list))

pokemon_page_links = [poke.find("a", class_="link__loadtItem")["href"] for poke in pokemon_list]
print(pokemon_page_links[:3])

ポケモンのページ数 1252
['/detail/0001', '/detail/0002', '/detail/0003']


In [ ]:
# ポケモンの各ページをクローリング。
base_url = lambda page_link: f"https://zukan.pokemon.co.jp/{page_link}"

for page in pokemon_page_links:
    url = base_url(page)
    output_path = poke_dir / (page.replace("/detail/", "")+".html")

    response = requests.get(url)
    response.text

    if response.status_code == 200:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print(f"ページを正常に保存しました: {output_path}")
    else:
        print(f"ステータスコードが200ではありません。ステータスコード：{response.status_code}")
        print(f"異常なページ：{page}")
    time.sleep(1)

ページを正常に保存しました: /content/data/pokemon/0001.html
ページを正常に保存しました: /content/data/pokemon/0002.html
ページを正常に保存しました: /content/data/pokemon/0003.html
ページを正常に保存しました: /content/data/pokemon/0003-1.html
ページを正常に保存しました: /content/data/pokemon/0003-2.html
ページを正常に保存しました: /content/data/pokemon/0004.html
ページを正常に保存しました: /content/data/pokemon/0005.html
ページを正常に保存しました: /content/data/pokemon/0006.html
ページを正常に保存しました: /content/data/pokemon/0006-1.html
ページを正常に保存しました: /content/data/pokemon/0006-2.html
ページを正常に保存しました: /content/data/pokemon/0006-3.html
ページを正常に保存しました: /content/data/pokemon/0007.html
ページを正常に保存しました: /content/data/pokemon/0008.html
ページを正常に保存しました: /content/data/pokemon/0009.html
ページを正常に保存しました: /content/data/pokemon/0009-1.html
ページを正常に保存しました: /content/data/pokemon/0009-2.html
ページを正常に保存しました: /content/data/pokemon/0010.html
ページを正常に保存しました: /content/data/pokemon/0011.html
ページを正常に保存しました: /content/data/pokemon/0012.html
ページを正常に保存しました: /content/data/pokemon/0012-1.html
ページを正常に保存しました: /content/data/pokemon/0013.ht

In [ ]:
# コピー元とコピー先のディレクトリを指定
source_dir = '/content/data'
destination_dir = '/content/drive/MyDrive/works/data/2501_pokemon'

# 目的のフォルダが存在しない場合は作成
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# フォルダごと再帰的にコピー
if os.path.exists(source_dir):
    shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
    print(f"'{source_dir}' の内容を '{destination_dir}' にコピーしました。")
else:
    print(f"'{source_dir}' が存在しません。")

print("すべてのファイルのコピーが完了しました！")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/data' の内容を '/content/drive/MyDrive/works/data/2501_pokemon' にコピーしました。
すべてのファイルのコピーが完了しました！


In [ ]:
# 各ファイルからポケモン図鑑を復元
pokemon_zukan = []
for page in os.listdir(poke_dir):
    with open(poke_dir / page, "r") as f:
        page_source = f.read()
    soup = BeautifulSoup(page_source, 'html.parser')
    script_tag = soup.find('script', {'id': 'json-data', 'type': 'application/json'})
    json_data = script_tag.string.strip()
    json_data = json.loads(json_data)
    pokemon_zukan.append(json_data["pokemon"])

In [6]:
# ポケモン図鑑をデータフレーム化。
df = pd.DataFrame(pokemon_zukan)
df.sort_values(by=["no", "sub"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

NameError: name 'pokemon_zukan' is not defined

In [ ]:
# ポケモン図鑑を保存
df.to_csv(poke_dir / "pokemon_zukan.csv", index=False)

# Google ドライブに保存
source_path = '/content/data/pokemon/pokemon_zukan.csv'
destination_path = '/content/drive/MyDrive/works/data/2501_pokemon/pokemon_zukan.csv'

shutil.copy(source_path, destination_path)
print(f'{source_path} を {destination_path} にコピーしました。')

/content/data/pokemon/pokemon_zukan.csv を /content/drive/MyDrive/works/data/2501_pokemon/pokemon_zukan.csv にコピーしました。


In [3]:
# ポケモン図鑑を読み込み。
df = pd.read_csv('/content/drive/MyDrive/works/data/2501_pokemon/pokemon_zukan.csv')
df.sample(3)

,no,sub,name,sub_name,area,omosa,takasa,sex,bunrui,tokusei_1,tokusei_2,type_1,type_2,text_1,text_2,text_3,spec_hp,spec_kougeki,spec_bougyo,spec_tokukou,spec_tokubou,spec_subayasa,sugata_text_flg,sugata_text,mega_flg,genshi_flg,kyodai_flg,image_l,image_m,image_s,zukan_no
875,716,0,ゼルネアス,NaN,6,215.0,3.0,0,せいめいポケモン,176,0,18,0,えいえんの いのちを わけあたえると いわれている。 じゅもくの すがたで １０００ねん ね...,あたまの ツノが なないろに かがやくとき えいえんの いのちを わけあたえると いわれてい...,NaN,8,8,6,8,6,6,0,NaN,0,0,0,https://zukan.pokemon.co.jp/zukan-api/up/image...,https://zukan.pokemon.co.jp/zukan-api/up/image...,https://zukan.pokemon.co.jp/zukan-api/up/image...,0716
561,450,0,カバルドン,NaN,4,300.0,2.0,1,じゅうりょうポケモン,92,0,9,0,意外と 怒りっぽい ポケモンで 口を 大きく 開けることで まわりに 強さを アピールする。...,大きく 口を 開けて 自分の 強さを アピール。 大量の 砂を 巻き上げて 攻撃する。 （『...,気は 短く 血の気 多し。 砂交じりの 旋風を 起こし 敵の威勢 挫きて 襲い掛かる。 （『...,7,7,7,4,5,3,0,NaN,0,0,0,https://zukan.pokemon.co.jp/zukan-api/up/image...,https://zukan.pokemon.co.jp/zukan-api/up/image...,https://zukan.pokemon.co.jp/zukan-api/up/image...,0450
1046,855,1,ポットデス,しんさくフォルム,8,0.4,0.2,0,こうちゃポケモン,55,0,14,0,入っている ティーポットによる 違いは ないが もとの 紅茶によって 気質が 変わるらしい。...,飲むと 危険だが 美味い。 危険を 恐れない グルメたちは 日々 ポットデスを 飲み続ける。...,NaN,4,4,4,8,7,5,0,NaN,0,0,0,https://zukan.pokemon.co.jp/zukan-api/up/image...,https://zukan.pokemon.co.jp/zukan-api/up/image...,https://zukan.pokemon.co.jp/zukan-api/up/image...,0855-1


In [15]:
# Dify アプリ開発用のベタ書き
# ポケモンクイズアプリのためのテキストを生成する
# ノートブック上にすべてを出力できないので、ファイル吐き出し
with open("output.txt", "w") as f:
    cols = ["name", "sub_name", "omosa", "takasa", "bunrui", "text_1", "text_2", "zukan_no", "image_s"]
    for i, (name, sub_name, omosa, takasa, bunrui, text_1, text_2, zukan_no, image_s) in df[cols].iterrows():

        full_name = name + "（" + sub_name + "）" if sub_name is not np.nan else name
        im = image_s.replace("https://zukan.pokemon.co.jp/zukan-api/up/images/index/", "")

        text = f'"""ポケモンの名前：{full_name}\n'
        text += f'重さ：{omosa}\n'
        text += f'高さ：{takasa}\n'
        text += f'分類：{bunrui}\n'
        text += f'説明1：{text_1}\n'
        text += f'説明2：{text_2}"""'
        print(f'    ("{full_name}", "{zukan_no}", "{im}", {text}\n    ),', file=f)

In [12]:
# Dify アプリ開発用のベタ書き
# ポケモンの名前と画像 URL と図鑑 URL
for i, (name, sub_name, zukan_no, image_s) in df[["name", "sub_name", "zukan_no", "image_s"]].iterrows():
    im = image_s.replace("https://zukan.pokemon.co.jp/zukan-api/up/images/index/", "")
    full_name = name + "（" + sub_name + "）" if sub_name is not np.nan else name
    print(f'    ("{full_name}", "{zukan_no}", "{im}"),')

    ("フシギダネ", "0001", "afa02eaba4c39820fc57f4e8abaeea80.png"),
    ("フシギソウ", "0002", "f78edf4c2bc037f4b23529edfcf9ddce.png"),
    ("フシギバナ", "0003", "cdce516974ae6a74e1b8b855644c5ce5.png"),
    ("メガフシギバナ", "0003-1", "5109c8badb2ef84e08a429e3f2535663.png"),
    ("フシギバナ（キョダイマックスのすがた）", "0003-2", "e0a15d7ddd7acb7c2d02c01ae8240b60.png"),
    ("ヒトカゲ", "0004", "8c45cf85becf84af9de3df62bb84a767.png"),
    ("リザード", "0005", "12012cd4efde1bff9b2692b403bd878b.png"),
    ("リザードン", "0006", "a686ec5e5f6b1d9731f20626a778a0bc.png"),
    ("メガリザードンＸ", "0006-1", "3e972438b54d530e82f4d63c6e4cb5bf.png"),
    ("メガリザードンＹ", "0006-2", "8356e6ff40700340223ea28b7b910987.png"),
    ("リザードン（キョダイマックスのすがた）", "0006-3", "697b801f8449411e018575ab43bc5f46.png"),
    ("ゼニガメ", "0007", "f1a0d49b1c09f66333fcb4ec28f9ea16.png"),
    ("カメール", "0008", "96a5203e00d4ba7197f5d438d9cdea0b.png"),
    ("カメックス", "0009", "bf747ef914143880c46016105ca155da.png"),
    ("メガカメックス", "0009-1", "1a01bf3e79832906ed08b3ac5ab7a22a.png"),
    ("カメッ